# Stored completions with Azure AI Foundry

Stored completions allow you to capture the conversation history from chat completions sessions to use as datasets for evaluations and fine-tuning.

> https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/stored-completions?tabs=python-key#stored-completions-api

> https://techcommunity.microsoft.com/blog/azure-ai-services-blog/introducing-model-distillation-in-azure-openai-service/4298627

In [1]:
import os
import sys

from dotenv import load_dotenv
from openai import AzureOpenAI
from datetime import datetime

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 18-Apr-2025 07:31:39


In [4]:
load_dotenv("azure.env")

True

In [5]:
aoai_client = AzureOpenAI(api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                          api_version="2025-02-01-preview",
                          azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT")
                         )

In [6]:
completion = aoai_client.chat.completions.create(
    model="gpt-4o",
    store=True,  # True
    metadata={
        "user": "admin",
        "category": "docs-test"
    },
    messages=[{
        "role":
        "system",
        "content":
        "Provide a clear and concise summary of the technical content, highlighting key concepts and their relationships. Focus on the main ideas and practical implications."
    }, {
        "role":
        "user",
        "content":
        "Ensemble methods combine multiple machine learning models to create a more robust and accurate predictor. Common techniques include bagging (training models on random subsets of data), boosting (sequentially training models to correct previous errors), and stacking (using a meta-model to combine base model predictions). Random Forests, a popular bagging method, create multiple decision trees using random feature subsets. Gradient Boosting builds trees sequentially, with each tree focusing on correcting the errors of previous trees. These methods often achieve better performance than single models by reducing overfitting and variance while capturing different aspects of the data."
    }])

print(completion.choices[0].message)

ChatCompletionMessage(content='Ensemble methods in machine learning are strategies used to improve prediction accuracy by combining multiple models. The key techniques include:\n\n1. **Bagging (Bootstrap Aggregating)**: This approach involves training multiple models on different random subsets of the data. It aims to reduce variance and avoid overfitting by averaging the predictions. Random Forests are a common application of bagging, where multiple decision trees are built using random subsets of features.\n\n2. **Boosting**: Unlike bagging, boosting trains models sequentially, with each model focusing on the errors made by its predecessor. This method reduces bias and increases model accuracy. Gradient Boosting is a specific boosting technique that enhances prediction by iteratively adding models to correct previous predictions.\n\n3. **Stacking (Stacked Generalization)**: Stacking combines predictions from various base models using a meta-model to improve overall performance. The m

In [7]:
response = aoai_client.chat.completions.list()

print(response.model_dump_json(indent=4))

{
    "data": [
        {
            "id": "chatcmpl-BIZkWKeTPC3jVafcGnxvtMvk7TJKQ",
            "choices": [
                {
                    "finish_reason": null,
                    "index": 0,
                    "logprobs": null,
                    "message": {
                        "content": "Ensemble methods are advanced techniques in machine learning that combine multiple models to enhance predictive accuracy and robustness. The key approaches include:\n\n1. **Bagging (Bootstrap Aggregating)**: This involves training models on different random subsets of the training data, aiming to reduce variance and overfitting. A prime example of bagging is Random Forests, where multiple decision trees are created using random subsets of features as well as data, providing stability and improved performance over individual trees.\n\n2. **Boosting**: This sequential technique focuses on training models to correct the errors made by previous models. Each subsequent model attempts t

In [9]:
response = aoai_client.chat.completions.list()

for index, item in enumerate(response.data, start=1):
    print(f"{index:05}: {item.id}")

00001: chatcmpl-BIZkWKeTPC3jVafcGnxvtMvk7TJKQ
00002: chatcmpl-BIZiJl2GSkQ0a1d9LG1AFlnfoZhyN
00003: chatcmpl-BIZgKYzU3LO0N08tV1RiVXWpxSpv7
00004: chatcmpl-BIZf0lObrIOpfFRrcc6hrHZD5mUbK
00005: chatcmpl-BIZbG29dttR70AmKyXEBciiD9tSd5
00006: chatcmpl-BIZah2M5drUpuo5krz2D0XVF64tsP
00007: chatcmpl-BIZaeJUEUbilyUbUgm8aZe6Q1sEFH
00008: chatcmpl-BIZabCWVQlnmR36WEnathtpghPOPa
00009: chatcmpl-BIZaXH2eBMArdY6re7mKXI8XHUKej
00010: chatcmpl-BIZa7vvTDa1NpGePfgtf76ReYoBr2
00011: chatcmpl-BIZa4WoZ6ZmTR4pDGnz1LVwRmO0ZE
00012: chatcmpl-BIZa0EbMj29zYDyOIavr2m09wEGAs
00013: chatcmpl-BIZZx4vOwsPt8dLKdM0K7oi9ZZkeG
00014: chatcmpl-BIZZsiHP0U9WatwSDay0TUJGFgG6W
00015: chatcmpl-BIZZpteC7DubJUWXuKXbWvzBz02y2


In [10]:
idexample = "chatcmpl-BIZZpteC7DubJUWXuKXbWvzBz02y2"

In [11]:
response = aoai_client.chat.completions.retrieve(idexample)

print(response.model_dump_json(indent=4))

{
    "id": "chatcmpl-BIZZpteC7DubJUWXuKXbWvzBz02y2",
    "choices": [
        {
            "finish_reason": null,
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "Ensemble methods enhance machine learning performance by combining multiple models to create a more accurate and robust predictor. The key techniques include:\n\n1. **Bagging (Bootstrap Aggregating):** It involves training multiple models on different random subsets of data to reduce variance and overfitting. Random Forests are a popular implementation, creating multiple decision trees using random subsets of features.\n\n2. **Boosting:** This technique builds models sequentially, each focusing on correcting errors made by previous models. Gradient Boosting is a common form, which incrementally improves predictions by addressing prior inaccuracies.\n\n3. **Stacking:** A meta-model is used to combine the predictions from various base models, leveraging their strengths

In [12]:
response = aoai_client.chat.completions.messages.list(idexample, limit=4)

print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "content": "Provide a clear and concise summary of the technical content, highlighting key concepts and their relationships. Focus on the main ideas and practical implications.",
      "refusal": null,
      "role": "system",
      "annotations": null,
      "audio": null,
      "function_call": null,
      "tool_calls": null,
      "id": "chatcmpl-BIZZpteC7DubJUWXuKXbWvzBz02y2-0"
    },
    {
      "content": "Ensemble methods combine multiple machine learning models to create a more robust and accurate predictor. Common techniques include bagging (training models on random subsets of data), boosting (sequentially training models to correct previous errors), and stacking (using a meta-model to combine base model predictions). Random Forests, a popular bagging method, create multiple decision trees using random feature subsets. Gradient Boosting builds trees sequentially, with each tree focusing on correcting the errors of previous trees. These methods often a

In [13]:
response = aoai_client.chat.completions.list()

print(response.model_dump_json(indent=4))

{
    "data": [
        {
            "id": "chatcmpl-BIZkWKeTPC3jVafcGnxvtMvk7TJKQ",
            "choices": [
                {
                    "finish_reason": null,
                    "index": 0,
                    "logprobs": null,
                    "message": {
                        "content": "Ensemble methods are advanced techniques in machine learning that combine multiple models to enhance predictive accuracy and robustness. The key approaches include:\n\n1. **Bagging (Bootstrap Aggregating)**: This involves training models on different random subsets of the training data, aiming to reduce variance and overfitting. A prime example of bagging is Random Forests, where multiple decision trees are created using random subsets of features as well as data, providing stability and improved performance over individual trees.\n\n2. **Boosting**: This sequential technique focuses on training models to correct the errors made by previous models. Each subsequent model attempts t

In [14]:
response = aoai_client.chat.completions.list()

for index, item in enumerate(response.data, start=1):
    print(f"{index:05}: {item.id}")

00001: chatcmpl-BIZkWKeTPC3jVafcGnxvtMvk7TJKQ
00002: chatcmpl-BIZiJl2GSkQ0a1d9LG1AFlnfoZhyN
00003: chatcmpl-BIZgKYzU3LO0N08tV1RiVXWpxSpv7
00004: chatcmpl-BIZf0lObrIOpfFRrcc6hrHZD5mUbK
00005: chatcmpl-BIZbG29dttR70AmKyXEBciiD9tSd5
00006: chatcmpl-BIZah2M5drUpuo5krz2D0XVF64tsP
00007: chatcmpl-BIZaeJUEUbilyUbUgm8aZe6Q1sEFH
00008: chatcmpl-BIZabCWVQlnmR36WEnathtpghPOPa
00009: chatcmpl-BIZaXH2eBMArdY6re7mKXI8XHUKej
00010: chatcmpl-BIZa7vvTDa1NpGePfgtf76ReYoBr2
00011: chatcmpl-BIZa4WoZ6ZmTR4pDGnz1LVwRmO0ZE
00012: chatcmpl-BIZa0EbMj29zYDyOIavr2m09wEGAs
00013: chatcmpl-BIZZx4vOwsPt8dLKdM0K7oi9ZZkeG
00014: chatcmpl-BIZZsiHP0U9WatwSDay0TUJGFgG6W
00015: chatcmpl-BIZZpteC7DubJUWXuKXbWvzBz02y2


In [15]:
response = aoai_client.chat.completions.delete(idexample)

print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-BIZZpteC7DubJUWXuKXbWvzBz02y2",
  "deleted": true,
  "object": "chat.completion.deleted"
}


In [16]:
response = aoai_client.chat.completions.list()

for index, item in enumerate(response.data, start=1):
    print(f"{index:05}: {item.id}")

00001: chatcmpl-BIZkWKeTPC3jVafcGnxvtMvk7TJKQ
00002: chatcmpl-BIZiJl2GSkQ0a1d9LG1AFlnfoZhyN
00003: chatcmpl-BIZgKYzU3LO0N08tV1RiVXWpxSpv7
00004: chatcmpl-BIZf0lObrIOpfFRrcc6hrHZD5mUbK
00005: chatcmpl-BIZbG29dttR70AmKyXEBciiD9tSd5
00006: chatcmpl-BIZah2M5drUpuo5krz2D0XVF64tsP
00007: chatcmpl-BIZaeJUEUbilyUbUgm8aZe6Q1sEFH
00008: chatcmpl-BIZabCWVQlnmR36WEnathtpghPOPa
00009: chatcmpl-BIZaXH2eBMArdY6re7mKXI8XHUKej
00010: chatcmpl-BIZa7vvTDa1NpGePfgtf76ReYoBr2
00011: chatcmpl-BIZa4WoZ6ZmTR4pDGnz1LVwRmO0ZE
00012: chatcmpl-BIZa0EbMj29zYDyOIavr2m09wEGAs
00013: chatcmpl-BIZZx4vOwsPt8dLKdM0K7oi9ZZkeG
00014: chatcmpl-BIZZsiHP0U9WatwSDay0TUJGFgG6W


In [17]:
len(response.data)

14